# reproduce data load in train_gpt2.py

discussion 1
- data loader self.current_position += n_tokens_in_batch or +1

In [1]:
import os 
import numpy as np
import torch

data_dir = 'edu_fineweb10B'
os.listdir(data_dir)

['edufineweb_train_000008.npy',
 'edufineweb_val_000000.npy',
 'edufineweb_train_000009.npy',
 'edufineweb_train_000001.npy',
 'edufineweb_train_000002.npy',
 'edufineweb_train_000003.npy',
 'edufineweb_train_000007.npy',
 'edufineweb_train_000006.npy',
 'edufineweb_train_000004.npy',
 'edufineweb_train_000010.npy',
 'edufineweb_train_000005.npy']

In [2]:
# load_tokens
data  = np.load(os.path.join(data_dir,'edufineweb_val_000000.npy'))
#data = data.astype(np.int32) 

data_tensor = torch.tensor(data, dtype = torch.long) 
data_tensor.shape

torch.Size([10000000])

In [3]:
def load_tokens(filename):

    data = np.load(filename)
    data = data.astype(np.int32)
    data_tensor = torch.tensor(data, dtype=torch.long)

    return data_tensor


In [4]:
class DataLoaderLite:

    def __init__(self, B, T, split, data_dir):

        self.B = B
        self.T = T
        self.split = split 
        assert split in {'train','val'}

        self.shards = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if split in filename]
        assert len(self.shards) > 0, "no shards"

        print(f"{len(self.shards)} shards for {split}")

        self.reset()


    def reset(self):

        self.current_shard = 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T 
        n_tokens_in_batch = B*T
        buf = self.tokens[self.current_position: self.current_position+n_tokens_in_batch+1]
        x = buf[:-1].view(B,T)
        y = buf[1:].view(B,T)

        self.current_position += n_tokens_in_batch #??
        
        # if the remaining is not enough to be a batch, skip the remaining and advance to the next shard
        if self.current_position + n_tokens_in_batch + 1 > len(self.tokens):
            self.current_shard = (self.current_shard + 1) % len(self.shards)
            self.current_position = 0
            self.tokens = load_tokens(self.shards[self.current_shard])

        return x, y


In [5]:
B = 2
T = 10
train_loader = DataLoaderLite(B, T, 'val', data_dir)

1 shards for val


In [6]:
train_loader.next_batch()

(tensor([[50256,   464, 13362, 12091,   198,  1890,   477,   262,  1842,    11],
         [19661,   290, 10731,   287, 12091,  2517,   268,   447,   247,    82]]),
 tensor([[  464, 13362, 12091,   198,  1890,   477,   262,  1842,    11, 19661],
         [  290, 10731,   287, 12091,  2517,   268,   447,   247,    82,  3835]]))

In [7]:
train_loader.next_batch()

(tensor([[ 3835,    11,   644,   484,   389,  1107,   546,   318,  4925,   290],
         [10404,    13, 20153,   286,  1807,   290,   262,  4925,   284,  3853]]),
 tensor([[   11,   644,   484,   389,  1107,   546,   318,  4925,   290, 10404],
         [   13, 20153,   286,  1807,   290,   262,  4925,   284,  3853,    13]]))